<a href="https://colab.research.google.com/github/HelioNeves/pdf-certificates-in-python/blob/master/certificate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create .pdf certificates in with Python and Google Drive


### Requirements


In [0]:
# RUN ONLY ONCE
!pip install -U -q PyDrive
!pip install PyPDF2
!pip install reportlab
!pip install --upgrade -q gspread

### Allow PyDrive mount your Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Check dataset Location
Indicate where in Google Drive spreadsheets are available:
ex.:

* drive/My Drive/UFCSPA/TEDxUFCSPA/Certificados/Participantes/

In [0]:
!ls 'drive/My Drive/spreadsheets_folder/'

#### Once you find your files, then set:

In [0]:
PATH = 'drive/My Drive/spreadsheets_folder/'
SPREADSHEET_NAME = 'name'
MODEL = "model_name.pdf"
CERTIFICATES_FOLDER = "Folder/"

### Load spreadsheet



In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open(SPREADSHEET_NAME).sheet1

# get_all_values gives a list of rows
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render
import pandas as pd
df = pd.DataFrame.from_records(rows)

### Transform PDF files and save in a folder

In [0]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io, time
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.rl_config import defaultPageSize

def pdf_print(name):
  PAGE_WIDTH  = defaultPageSize[1]


  packet = io.BytesIO()
  # create a new PDF with Reportlab Lucas Costa dos Santos
  can = canvas.Canvas(packet)

  ''' SET FONT, FONT SIZE AND TEXT HEIGHT 
  '''
  text = name
  text_width = stringWidth(text, 'Helvetica-Bold', 18)
  height = 340 # wherever you want your text to appear
  can.setFont('Helvetica-Bold', 18) # Change Font, Font-size, etc
  can.drawString((PAGE_WIDTH - text_width) / 2.0, height, text)
  can.save()
  '''
  '''
  # move to the beginning of the StringIO buffer
  packet.seek(0)
  new_pdf = PdfFileReader(packet)
  # read your existing PDF
  existing_pdf = PdfFileReader(open(PATH+MODEL, "rb"))
  output = PdfFileWriter()
  page = existing_pdf.getPage(0)
  page.mergePage(new_pdf.getPage(0))
  output.addPage(page)
  # finally, write "output" to a real file
  print("Published the certificate: "+name)
  outputStream = open(PATH+CERTIFICATES_FOLDER+name+".pdf", "wb")
  output.write(outputStream)
  outputStream.close()

### Let's Play 

In [0]:
for index, row in df.iterrows():
  pdf_print(row[0].upper())